In [1]:
import pandas as pd
import numpy as np

In [2]:
#pulling in job csv
jobs_ = "data/Job Postings.csv"
jobs_df = pd.read_csv(jobs_)

In [3]:
#pulling in covid/merchant csv
covidmerchant_ = "output_files/covidmerchant.csv"
covidmerchant_df = pd.read_csv(covidmerchant_)

In [4]:
#merging jobs/covid/merchants
covidmerchantjobs_df = pd.merge(covidmerchant_df,jobs_df, on=['cityid'], how='outer', suffixes=('_cvm','_jobs'))

In [5]:
#changing cityname to City Name to match cost of living
cv19bizjob_df = covidmerchantjobs_df.rename(columns={"cityname": "City Name"})

In [6]:
#pulling in living csv
living_ = "output_files/cost_of_living.csv"
living_df = pd.read_csv(living_)

In [7]:
#adding 1 bedroom true cost (if needed)
living_df["1 Bedroom True Cost"]=living_df.iloc[:,[1,3]].sum(axis=1)

#adding 3 bedroom true cost (if needed)
living_df["3 Bedroom True Cost"]=living_df.iloc[:,[2,3]].sum(axis=1)

#adding Average true cost (if needed)
living_df["Avg True Cost"]=living_df.iloc[:,[4,5]].mean(axis=1).round(2)

In [8]:
#merging all info together - covid/merhcnat/jobs/living
#replacing Nan values with "Data Not Available"
allinfo_df = pd.merge(cv19bizjob_df,living_df, on=['City Name'], how='outer')
allinfo_df = allinfo_df.replace({np.NaN: "Data Not Available"})

In [9]:
#keeping cityid lat lon citypop merchantsall(change to%) newcaserate bgpost avetrucost
allinfo_df = allinfo_df.drop(['year_cvm',
                        'month_cvm',
                        'day',
                        'month_jobs',
                        'day_endofweek',
                        'Cost of 1 Bedroom Appartment',
                        'Cost of 3 Bedroom Appartment',
                        'Utilities Per Month',
                        '1 Bedroom True Cost',
                        '3 Bedroom True Cost',
                        'Merchants Difference',
                        'Difference plus 1',
                        'year_jobs',
                        'year_month'], axis = 1)

In [10]:
#pulling in covid/merchant csv
blsgeo_ = "output_files/BLS_GEO_OCCUPATION_ALL_df_merge.csv"
blsgeo_df = pd.read_csv(blsgeo_,index_col=0)

In [11]:
#keeping geocitystate mwe occtypecode occtype skilllevel howpaid cityid
blsgeo_df = blsgeo_df.drop(['year',
                        'cityname',
                       'stateabbrev',
                        'statefips',
                        'lat',
                        'lon',
                        'city_pop2019',
                        'statename'], axis = 1)

In [12]:
#using a pivot table
ndf = blsgeo_df.pivot(index = "cityid", columns = 'MWE_Occupation Type', values = 'MWE')

In [13]:
#merging last df with pivot df
alldfs = pd.merge(allinfo_df,ndf, on=['cityid'], how='outer')

In [14]:
#replacing all values not accounted for as "Data Not Available"
alldfs = alldfs.replace({'Not Found': "Data Not Available", "-": "Data Not Available"})

In [15]:
#formatting the merchants_all column to percentage
alldfs['merchants_all'] = pd.Series(["{0:.2f}%".format(val * 100) for val in alldfs['merchants_all']], index = alldfs.index)

In [16]:
#displaying df
alldfs

,cityid,City Name,stateabbrev,lat,lon,city_pop2019,merchants_all,new_case_rate,bg_posts,Avg True Cost,...,"Installation, maintenance, and repair occupations",Legal occupations,"Life, physical, and social science occupations",Management occupations,Office and administrative support occupations,Personal care and service occupations,Production occupations,Protective service occupations,Sales and related occupations,Transportation and material moving occupations
0,1,Los Angeles,CA,34.05,-118.24,10039107,-33.30%,18.60,0.05670,3440.32,...,27.69,66.66,43.17,65.30,22.75,17.66,19.49,30.72,27.44,20.55
1,2,New York City,NY,40.71,-74.01,8336817,-42.00%,49.30,0.15300,4916.34,...,26.79,62.45,37.96,76.33,23.84,18.49,20.70,30.25,34.97,22.50
2,3,Chicago,IL,41.88,-87.63,5150233,-39.60%,11.00,0.06270,2905.82,...,25.95,86.08,36.51,61.09,22.17,16.88,19.94,29.72,29.11,13.76
3,4,Houston,TX,29.76,-95.37,4713325,-56.70%,14.50,0.06590,2096.92,...,24.63,Data Not Available,44.73,61.53,20.21,15.28,22.11,22.10,26.62,14.26
4,5,Phoenix,AZ,33.45,-112.07,4485414,-29.30%,22.10,0.06770,1842.06,...,23.82,57.36,30.27,54.71,19.94,Data Not Available,19.41,24.89,25.27,21.29
5,6,San Diego,CA,32.72,-117.16,3338330,-36.10%,21.80,0.00768,3097.96,...,25.91,60.48,41.27,64.72,21.87,17.89,21.06,29.73,25.81,20.03
6,7,Dallas,TX,32.78,-96.80,2635516,-52.50%,13.70,0.01900,2445.33,...,24.65,26.05,Data Not Available,62.31,19.97,15.45,18.69,24.69,26.93,21.91
7,8,Las Vegas,NV,36.17,-115.14,2266715,-28.40%,12.10,0.01050,1778.28,...,25.47,50.54,33.14,55.35,18.76,14.68,18.75,24.81,22.75,12.19
8,9,Seattle,WA,47.61,-122.33,2252782,-25.90%,9.70,0.13600,3233.79,...,29.65,53.58,39.88,67.91,23.73,20.94,25.56,31.77,30.97,25.32
9,10,Fort Worth,TX,32.76,-97.33,2102515,-52.40%,19.40,0.06190,1634.36,...,24.65,26.05,Data Not Available,62.31,19.97,15.45,18.69,24.69,26.93,21.91


In [17]:
#Matt did stuffs

# x = [occname for occname in alldfs.columns if "occupations" in occname]
# for i in x:
#     print(i)
    
# occ = input("Whats your job?")
# alldfs[alldfs["MWE occ type"]==occ]

In [18]:
#len(x)